In [1]:
!nvidia-smi

zsh:1: command not found: nvidia-smi


# 匯入函式庫

In [2]:
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader


# CPU/GPU、自定義資料集、模型、訓練函數

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
def SetSeed(myseed):
    # Python random module
    random.seed(myseed)
    # Numpy
    np.random.seed(myseed)
    # Torch
    torch.manual_seed(myseed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(myseed)
        torch.cuda.manual_seed_all(myseed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [5]:
class TimeSeriesDataset(Dataset):
    def __init__(self, X, WindowSize):
        X = np.expand_dims(X, 1)
        self.X = torch.from_numpy(X)
        self.WindowSize = WindowSize
        
    def __len__(self):
        return len(self.X) - self.WindowSize

    def __getitem__(self, idx):
        return (self.X[idx:idx+self.WindowSize], self.X[idx+self.WindowSize])
        # return (X = [seqs, features], y)

In [6]:
class LSTM(nn.Module):
    def __init__(self, num_layers, hidden_size):
        super().__init__()
        
        self.Input_HiddenLayer = nn.LSTM(input_size=1, hidden_size=hidden_size, num_layers=num_layers)

        self.OutputLayer = nn.Linear(hidden_size, 1)

    def forward(self, input):
        # input.shape = [BatchSize, WindowSize, 1]
        input = input.permute(1, 0, 2)
        # input.shape = [WindowSize, BatchSize, 1]
        hidden, _ = self.Input_HiddenLayer(input)
        # hidden.shape = [WindowSize, BatchSize, HiddenSize]
        hidden = hidden[-1]
        # hidden.shape = [BatchSize, HiddenSize]
        output = self.OutputLayer(hidden)
        
        return output

In [7]:
class Self_Attention(nn.Module):
    def __init__(self, num_layers, hidden_size):
        super().__init__()
        self.num_layers = num_layers
        
        self.Input_First_HiddenLayer = nn.MultiheadAttention(embed_dim=hidden_size, num_heads=1)
        self.Query_Key_Value_1 = nn.ModuleList([nn.Linear(1, hidden_size), nn.Linear(1, hidden_size), nn.Linear(1, hidden_size)])

        self.Second_And_Following_HiddenLayer = nn.ModuleList([nn.MultiheadAttention(embed_dim=hidden_size, num_heads=1) for i in range(num_layers - 1)])
        self.List_of_Query_Key_Value = nn.ModuleList([nn.ModuleList([nn.Linear(hidden_size, hidden_size) for q_k_v in range(3)]) for i in range(num_layers - 1)])

        self.OutputLayer = nn.Linear(hidden_size, 1)

    def forward(self, input):
        # input.shape = [BatchSize, WindowSize, 1]
        input = input.permute(1, 0, 2)
        # input.shape = [WindowSize, BatchSize, 1]
        Query1 = self.Query_Key_Value_1[0](input)
        Key1 = self.Query_Key_Value_1[1](input)
        Value1 = self.Query_Key_Value_1[2](input)
        hidden, _ = self.Input_First_HiddenLayer(Query1, Key1, Value1)
        # hidden.shape = [WindowSize, BatchSize, HiddenSize]
        if self.num_layers > 1:
            for i, Second_And_Following_HiddenLayer in enumerate(self.Second_And_Following_HiddenLayer):
                Query = self.List_of_Query_Key_Value[i][0](hidden)
                Key = self.List_of_Query_Key_Value[i][1](hidden)
                Value = self.List_of_Query_Key_Value[i][2](hidden)
                hidden, _ = Second_And_Following_HiddenLayer(Query, Key, Value)
        # hidden.shape = [WindowSize, BatchSize, HiddenSize]
        hidden = hidden[-1]
        # hidden.shape = [BatchSize, HiddenSize]
        output = self.OutputLayer(hidden)
        
        return output

In [8]:
def train_under_config(model_type,
                       forex_data,
                       length_input_sequence,
                       num_epochs,
                       num_hidden_layers,
                       num_hidden_sizes,
                       batch_sizes,
                       device):
    '''
    model_type: LSTM or Self_Attention,
    forex_data,
    length_input_sequence,
    num_epochs,
    learning_rate,
    num_hidden_layers,
    num_hidden_sizes,
    batch_sizes,
    device
    '''
    # setseed
    SetSeed(9527)
    # dataset
    training_data = forex_data.loc['1981-01-01':'2016-12-31'] # training_data = training set + validation set
    training_dataset = TimeSeriesDataset(training_data, length_input_sequence)
    # dataloader
    training_dataloader = DataLoader(training_dataset, batch_size=batch_sizes, shuffle=True)
    # model
    model = model_type(num_hidden_layers, num_hidden_sizes).double()
    # criterion & optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters())
    # training
    model.to(device)
    model.train()
    for epoch in range(num_epochs):
        for X, y in training_dataloader:
            optimizer.zero_grad()
            X, y = X.to(device), y.to(device)
            ypred = model(X)
            loss = criterion(ypred, y)
            loss.backward()
            optimizer.step()
    
    return model

In [54]:
# residue pred
def ypred(data, length_input_sequence, model, device):
    '''
    data,
    length_input_sequence,
    model,
    device
    '''
    # dataset
    testing_start_index = len(data.loc['1981-01-01':'2016-12-31']) - length_input_sequence
    testing_end_index = len(data.loc['1981-01-01':'2020-12-31'])
    testing_data = data[testing_start_index:testing_end_index]
    testing_dataset = TimeSeriesDataset(testing_data, length_input_sequence)
    # dataloader
    data_len = len(testing_dataset)
    testing_dataloader = DataLoader(testing_dataset, batch_size=data_len, shuffle=False)
    # evaluating
    model.to(device)
    model.eval()
    for X, y in  testing_dataloader:
        X, y = X.to(device), y.to(device)
        with torch.no_grad():
            ypred = model(X)
            ypred = np.array(ypred.cpu()).flatten()
    return ypred

In [36]:
def prediction_on_testset(model_type, linear_data, residue_data, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs):
    model = train_under_config(model_type,
                               residue_data,
                               length_input_sequence,
                               num_epochs,
                               num_hidden_layers,
                               num_hidden_sizes,
                               batch_sizes,
                               device)
    model_type_linear_pred = np.array(linear_data.loc['2017-01-01':'2020-12-31'])
    model_type_residue_pred = ypred(residue_data, length_input_sequence, model, device)
    model_type_pred = model_type_linear_pred + model_type_residue_pred
    return model_type_pred

In [42]:
# random walk prediction
def rw_pred(testing_data):
    return np.zeros(len(testing_data))

In [12]:
def root_mean_squared_error(ypred, ytrue):
    squared_error = (ypred - ytrue) ** 2
    mean_squared_error = sum(squared_error) / len(squared_error)
    return np.sqrt(mean_squared_error)

In [13]:
def mean_absolute_error(ypred, ytrue):
    absolute_error = abs(ypred - ytrue)
    return sum(absolute_error) / len(absolute_error)

In [14]:
def direction_accuracy(ypred, ytrue):
    ypred_elementwise_dot_ytrue = ypred * ytrue
    same_direction_rate = sum(ypred_elementwise_dot_ytrue > 0) / len(ypred_elementwise_dot_ytrue)
    return same_direction_rate

# 匯入訓練資料（訓練加驗證集）訓練，產生預測資料並計算模型的 RMSE, MAE, DA

## Recursive window

In [28]:
recursive_linear = pd.read_csv('./recursive_linear_prediction.csv', index_col=0)
recursive_linear = recursive_linear.astype('float64')
recursive_linear.index = pd.to_datetime(recursive_linear.index)

cad_recursive_linear = recursive_linear.iloc[:,0]
aud_recursive_linear = recursive_linear.iloc[:,1]
gbp_recursive_linear = recursive_linear.iloc[:,2]

recursive_linear

,cad_recursive_linear_prediction,aud_recursive_linear_prediction,gbp_recursive_linear_prediction
1981-01-02,-0.000018,-0.000058,-0.000071
1981-01-05,-0.000018,-0.000058,-0.000167
1981-01-06,-0.000018,-0.000058,0.000401
1981-01-07,-0.000018,-0.000058,0.000029
1981-01-08,-0.000018,-0.000058,-0.000221
...,...,...,...
2020-12-25,-0.000012,-0.000047,0.000109
2020-12-28,-0.000012,-0.000047,-0.000048
2020-12-29,-0.000012,-0.000048,-0.000323
2020-12-30,-0.000012,-0.000047,0.000074


In [29]:
recursive_residue = pd.read_csv('./recursive_linear_residue.csv', index_col=0)
recursive_residue = recursive_residue.astype('float64')
recursive_residue.index = pd.to_datetime(recursive_residue.index)

cad_recursive_residue = recursive_residue.iloc[:,0]
aud_recursive_residue = recursive_residue.iloc[:,1]
gbp_recursive_residue = recursive_residue.iloc[:,2]

recursive_residue

,cad_recursive_linear_residue,aud_recursive_linear_residue,gbp_recursive_linear_residue
1981-01-02,-0.000221,-0.000959,-0.002864
1981-01-05,0.005142,0.003863,0.014132
1981-01-06,0.000374,0.003681,0.002493
1981-01-07,0.000255,0.000058,-0.004580
1981-01-08,0.000018,-0.002047,-0.003102
...,...,...,...
2020-12-25,-0.000117,0.001100,0.000038
2020-12-28,0.001426,-0.003377,-0.007949
2020-12-29,0.002194,0.003604,0.004107
2020-12-30,0.005254,0.010510,0.008920


### LSTM

In [31]:
model_type = LSTM

#### cad

In [32]:
cad_recursive_linear

1981-01-02   -0.000018
1981-01-05   -0.000018
1981-01-06   -0.000018
1981-01-07   -0.000018
1981-01-08   -0.000018
                ...   
2020-12-25   -0.000012
2020-12-28   -0.000012
2020-12-29   -0.000012
2020-12-30   -0.000012
2020-12-31   -0.000011
Name: cad_recursive_linear_prediction, Length: 10435, dtype: float64

In [33]:
cad_recursive_residue

1981-01-02   -0.000221
1981-01-05    0.005142
1981-01-06    0.000374
1981-01-07    0.000255
1981-01-08    0.000018
                ...   
2020-12-25   -0.000117
2020-12-28    0.001426
2020-12-29    0.002194
2020-12-30    0.005254
2020-12-31    0.001413
Name: cad_recursive_linear_residue, Length: 10435, dtype: float64

In [58]:
# optim hyperparameters obtained by grid search
length_input_sequence = 5
num_hidden_layers = 2
num_hidden_sizes = 1600
batch_sizes = 16
num_epochs = 5

In [59]:
cad_recursive_LSTM_pred = prediction_on_testset(model_type, cad_recursive_linear, cad_recursive_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

#### aud

In [ ]:
aud_recursive_linear

In [ ]:
aud_recursive_residue

In [ ]:
# optim hyperparameters obtained by grid search
length_input_sequence = 10
num_hidden_layers = 3
num_hidden_sizes = 1600
batch_sizes = 128
num_epochs = 15

In [ ]:
aud_recursive_LSTM_pred = prediction_on_testset(model_type, aud_recursive_linear, aud_recursive_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

#### gbp

In [ ]:
gbp_recursive_linear

In [ ]:
gbp_recursive_residue

In [ ]:
# optim hyperparameters obtained by grid search
length_input_sequence = 5
num_hidden_layers = 2
num_hidden_sizes = 25
batch_sizes = 256
num_epochs = 5

In [ ]:
gbp_recursive_LSTM_pred = prediction_on_testset(model_type, gbp_recursive_linear, gbp_recursive_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

### Self-Attention

In [17]:
model_type = Self_Attention

#### cad

In [18]:
cad_recursive_linear

1981-01-02   -0.000018
1981-01-05   -0.000018
1981-01-06   -0.000018
1981-01-07   -0.000018
1981-01-08   -0.000018
                ...   
2020-12-25   -0.000012
2020-12-28   -0.000012
2020-12-29   -0.000012
2020-12-30   -0.000012
2020-12-31   -0.000011
Name: cad_recursive_linear_prediction, Length: 10435, dtype: float64

In [19]:
cad_recursive_residue

1981-01-02   -0.000221
1981-01-05    0.005142
1981-01-06    0.000374
1981-01-07    0.000255
1981-01-08    0.000018
                ...   
2020-12-25   -0.000117
2020-12-28    0.001426
2020-12-29    0.002194
2020-12-30    0.005254
2020-12-31    0.001413
Name: cad_recursive_linear_residue, Length: 10435, dtype: float64

In [20]:
# optim hyperparameters obtained by grid search
length_input_sequence = 5
num_hidden_layers = 1
num_hidden_sizes = 200
batch_sizes = 256
num_epochs = 15

In [ ]:
cad_recursive_Self_Attention_pred = prediction_on_testset(model_type, cad_recursive_linear, cad_recursive_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

#### aud

In [ ]:
aud_recursive_linear

In [ ]:
aud_recursive_residue

In [ ]:
# optim hyperparameters obtained by grid search
length_input_sequence = 10
num_hidden_layers = 1
num_hidden_sizes = 800
batch_sizes = 16
num_epochs = 30

In [ ]:
aud_recursive_Self_Attention_pred = prediction_on_testset(model_type, aud_recursive_linear, aud_recursive_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

#### gbp

In [ ]:
gbp_recursive_linear

In [ ]:
gbp_recursive_residue

In [ ]:
# optim hyperparameters obtained by grid search
length_input_sequence = 5
num_hidden_layers = 1
num_hidden_sizes = 200
batch_sizes = 256
num_epochs = 20

In [ ]:
gbp_recursive_Self_Attention_pred = prediction_on_testset(model_type, gbp_recursive_linear, gbp_recursive_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

## Rolling window

In [23]:
rolling_linear = pd.read_csv('./rolling_linear_prediction.csv', index_col=0)
rolling_linear = rolling_linear.astype('float64')
rolling_linear.index = pd.to_datetime(rolling_linear.index)

cad_rolling_linear = rolling_linear.iloc[:,0]
aud_rolling_linear = rolling_linear.iloc[:,1]
gbp_rolling_linear = rolling_linear.iloc[:,2]

rolling_linear

,cad_rolling_linear_prediction,aud_rolling_linear_prediction,gbp_rolling_linear_prediction
2017-01-02,-0.000018,-0.000058,0.000147
2017-01-03,-0.000018,-0.000058,-0.000237
2017-01-04,-0.000018,-0.000058,-0.000182
2017-01-05,-0.000017,-0.000057,0.000165
2017-01-06,-0.000016,-0.000056,0.000192
...,...,...,...
2020-12-25,-0.000002,-0.000015,0.000171
2020-12-28,-0.000002,-0.000014,0.000020
2020-12-29,-0.000002,-0.000014,-0.000245
2020-12-30,-0.000002,-0.000014,0.000138


In [24]:
rolling_residue = pd.read_csv('./rolling_linear_residue.csv', index_col=0)
rolling_residue = rolling_residue.astype('float64')
rolling_residue.index = pd.to_datetime(rolling_residue.index)

cad_rolling_residue = rolling_residue.iloc[:,0]
aud_rolling_residue = rolling_residue.iloc[:,1]
gbp_rolling_residue = rolling_residue.iloc[:,2]

rolling_residue

,cad_rolling_linear_residue,aud_rolling_linear_residue,gbp_rolling_linear_residue
2017-01-02,-0.000251,-0.003278,-0.005184
2017-01-03,0.001227,0.004918,-0.003108
2017-01-04,0.009506,0.008884,0.007186
2017-01-05,0.005720,0.007581,0.007595
2017-01-06,-0.000910,-0.004999,-0.010878
...,...,...,...
2020-12-25,-0.000126,0.001068,-0.000024
2020-12-28,0.001416,-0.003410,-0.008017
2020-12-29,0.002184,0.003571,0.004029
2020-12-30,0.005244,0.010477,0.008856


### LSTM

In [ ]:
model_type = LSTM

#### cad

In [ ]:
cad_rolling_linear

In [ ]:
cad_rolling_residue

In [ ]:
# optim hyperparameters obtained by grid search
length_input_sequence = 5
num_hidden_layers = 2
num_hidden_sizes = 1600
batch_sizes = 16
num_epochs = 5

In [ ]:
cad_rolling_LSTM_pred = prediction_on_testset(model_type, cad_rolling_linear, cad_rolling_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

#### aud

In [ ]:
aud_rolling_linear

In [ ]:
aud_rolling_residue

In [ ]:
# optim hyperparameters obtained by grid search
length_input_sequence = 5
num_hidden_layers = 2
num_hidden_sizes = 1600
batch_sizes = 16
num_epochs = 5

In [ ]:
aud_rolling_LSTM_pred = prediction_on_testset(model_type, aud_rolling_linear, aud_rolling_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

#### gbp

In [ ]:
gbp_rolling_linear

In [ ]:
gbp_rolling_residue

In [ ]:
# optim hyperparameters obtained by grid search
length_input_sequence = 5
num_hidden_layers = 2
num_hidden_sizes = 1600
batch_sizes = 16
num_epochs = 5

In [ ]:
gbp_rolling_LSTM_pred = prediction_on_testset(model_type, gbp_rolling_linear, gbp_rolling_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

### Self-Attention

In [ ]:
model_type = Self_Attention

#### cad

In [ ]:
cad_rolling_linear

In [ ]:
cad_rolling_residue

In [ ]:
# optim hyperparameters obtained by grid search
length_input_sequence = 5
num_hidden_layers = 2
num_hidden_sizes = 1600
batch_sizes = 16
num_epochs = 5

In [ ]:
cad_rolling_Self_Attention_pred = prediction_on_testset(model_type, cad_rolling_linear, cad_rolling_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

#### aud

In [ ]:
aud_rolling_linear

In [ ]:
aud_rolling_residue

In [ ]:
# optim hyperparameters obtained by grid search
length_input_sequence = 5
num_hidden_layers = 2
num_hidden_sizes = 1600
batch_sizes = 16
num_epochs = 5

In [ ]:
aud_rolling_Self_Attention_pred = prediction_on_testset(model_type, aud_rolling_linear, aud_rolling_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

#### gbp

In [ ]:
gbp_rolling_linear

In [ ]:
gbp_rolling_residue

In [ ]:
# optim hyperparameters obtained by grid search
length_input_sequence = 5
num_hidden_layers = 2
num_hidden_sizes = 1600
batch_sizes = 16
num_epochs = 5

In [ ]:
gbp_rolling_Self_Attention_pred = prediction_on_testset(model_type, gbp_rolling_linear, gbp_rolling_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

# 輸出模型預測與比較

## 輸出模型預測

In [ ]:
LSTM_recursive_prediction = pd.concat([cad_recursive_LSTM_pred, aud_recursive_LSTM_pred, gbp_recursive_LSTM_pred], axis=1)
LSTM_recursive_prediction.columns = ['cad_recursive_LSTM_pred', 'aud_recursive_LSTM_pred', 'gbp_recursive_LSTM_pred']
LSTM_recursive_prediction.to_csv('./LSTM_recursive_prediction.csv')
LSTM_recursive_prediction

In [ ]:
LSTM_rolling_prediction = pd.concat([cad_rolling_LSTM_pred, aud_rolling_LSTM_pred, gbp_rolling_LSTM_pred], axis=1)
LSTM_rolling_prediction.columns = ['cad_rolling_LSTM_pred', 'aud_rolling_LSTM_pred', 'gbp_rolling_LSTM_pred']
LSTM_rolling_prediction.to_csv('./LSTM_rolling_prediction.csv')
LSTM_rolling_prediction

In [ ]:
Self_Attention_recursive_prediction = pd.concat([cad_recursive_Self_Attention_pred, aud_recursive_Self_Attention_pred, gbp_recursive_Self_Attention_pred], axis=1)
Self_Attention_recursive_prediction.columns = ['cad_recursive_Self_Attention_pred', 'aud_recursive_Self_Attention_pred', 'gbp_recursive_Self_Attention_pred']
Self_Attention_recursive_prediction.to_csv('./Self_Attention_recursive_prediction.csv')
Self_Attention_recursive_prediction

In [ ]:
Self_Attention_rolling_prediction = pd.concat([cad_rolling_Self_Attention_pred, aud_rolling_Self_Attention_pred, gbp_rolling_Self_Attention_pred], axis=1)
Self_Attention_rolling_prediction.columns = ['cad_rolling_Self_Attention_pred', 'aud_rolling_Self_Attention_pred', 'gbp_rolling_Self_Attention_pred']
Self_Attention_rolling_prediction.to_csv('./Self_Attention_rolling_prediction.csv')
Self_Attention_rolling_prediction

## 載入實際匯率變動率（取對數後一階差分）

In [40]:
groundtruth = pd.read_excel('./CAD_AUD_GBP.xlsx', index_col=0, skiprows=3)
groundtruth = groundtruth.iloc[1:,:]
groundtruth = groundtruth.astype('float64')
groundtruth.index = pd.to_datetime(groundtruth.index)
groundtruth = np.log(groundtruth)
groundtruth = groundtruth.diff().dropna()

groundtruth = groundtruth.loc['2017-01-01':'2020-12-31']
cad_groundtruth = groundtruth.iloc[:,0]
aud_groundtruth = groundtruth.iloc[:,1]
gbp_groundtruth = groundtruth.iloc[:,2]

groundtruth

,CADUSD Curncy,AUDUSD Curncy,GBPUSD Curncy
2017-01-02,-0.000269,-0.003335,-0.005037
2017-01-03,0.001209,0.004860,-0.003345
2017-01-04,0.009488,0.008826,0.007003
2017-01-05,0.005703,0.007523,0.007760
2017-01-06,-0.000926,-0.005055,-0.010686
...,...,...,...
2020-12-25,-0.000129,0.001052,0.000148
2020-12-28,0.001414,-0.003425,-0.007996
2020-12-29,0.002182,0.003556,0.003784
2020-12-30,0.005242,0.010463,0.008994


## 模型比較

### Random walk pred

In [60]:
random_walk_pred = rw_pred(cad_groundtruth)

cad_random_walk_RMSE = root_mean_squared_error(random_walk_pred, cad_groundtruth)
cad_random_walk_MAE = mean_absolute_error(random_walk_pred, cad_groundtruth)
cad_random_walk_DA = 0.5
aud_random_walk_RMSE = root_mean_squared_error(random_walk_pred, aud_groundtruth)
aud_random_walk_MAE = mean_absolute_error(random_walk_pred, aud_groundtruth)
aud_random_walk_DA = 0.5
gbp_random_walk_RMSE = root_mean_squared_error(random_walk_pred, gbp_groundtruth)
gbp_random_walk_MAE = mean_absolute_error(random_walk_pred, gbp_groundtruth)
gbp_random_walk_DA = 0.5

In [61]:
cad_random_walk_RMSE, cad_random_walk_MAE, cad_random_walk_DA

(0.004365277893580917, 0.0032165540648211085, 0.5)

In [62]:
aud_random_walk_RMSE, aud_random_walk_MAE, aud_random_walk_DA

(0.005655793284191722, 0.004162599536671182, 0.5)

In [63]:
gbp_random_walk_RMSE, gbp_random_walk_MAE, gbp_random_walk_DA

(0.0056882604301849125, 0.004202877474568311, 0.5)

### Recursive LSTM pred

In [ ]:
cad_recursive_LSTM_RMSE = root_mean_squared_error(cad_recursive_LSTM_pred, cad_groundtruth)
cad_recursive_LSTM_MAE = mean_absolute_error(cad_recursive_LSTM_pred, cad_groundtruth)
cad_recursive_LSTM_DA = direction_accuracy(cad_recursive_LSTM_pred, cad_groundtruth)
aud_recursive_LSTM_RMSE = root_mean_squared_error(aud_recursive_LSTM_pred, aud_groundtruth)
aud_recursive_LSTM_MAE = mean_absolute_error(aud_recursive_LSTM_pred, aud_groundtruth)
aud_recursive_LSTM_DA = direction_accuracy(aud_recursive_LSTM_pred, aud_groundtruth)
gbp_recursive_LSTM_RMSE = root_mean_squared_error(gbp_recursive_LSTM_pred, gbp_groundtruth)
gbp_recursive_LSTM_MAE = mean_absolute_error(gbp_recursive_LSTM_pred, gbp_groundtruth)
gbp_recursive_LSTM_DA = direction_accuracy(gbp_recursive_LSTM_pred, gbp_groundtruth)

In [ ]:
cad_recursive_LSTM_RMSE, cad_recursive_LSTM_MAE, cad_recursive_LSTM_DA

In [ ]:
aud_recursive_LSTM_RMSE, aud_recursive_LSTM_MAE, aud_recursive_LSTM_DA

In [ ]:
gbp_recursive_LSTM_RMSE, gbp_recursive_LSTM_MAE, gbp_recursive_LSTM_DA

### Rolling LSTM pred

In [ ]:
cad_rolling_LSTM_RMSE = root_mean_squared_error(cad_rolling_LSTM_pred, cad_groundtruth)
cad_rolling_LSTM_MAE = mean_absolute_error(cad_rolling_LSTM_pred, cad_groundtruth)
cad_rolling_LSTM_DA = direction_accuracy(cad_rolling_LSTM_pred, cad_groundtruth)
aud_rolling_LSTM_RMSE = root_mean_squared_error(aud_rolling_LSTM_pred, aud_groundtruth)
aud_rolling_LSTM_MAE = mean_absolute_error(aud_rolling_LSTM_pred, aud_groundtruth)
aud_rolling_LSTM_DA = direction_accuracy(aud_rolling_LSTM_pred, aud_groundtruth)
gbp_rolling_LSTM_RMSE = root_mean_squared_error(gbp_rolling_LSTM_pred, gbp_groundtruth)
gbp_rolling_LSTM_MAE = mean_absolute_error(gbp_rolling_LSTM_pred, gbp_groundtruth)
gbp_rolling_LSTM_DA = direction_accuracy(gbp_rolling_LSTM_pred, gbp_groundtruth)

In [ ]:
cad_rolling_LSTM_RMSE, cad_rolling_LSTM_MAE, cad_rolling_LSTM_DA

In [ ]:
aud_rolling_LSTM_RMSE, aud_rolling_LSTM_MAE, aud_rolling_LSTM_DA

In [ ]:
gbp_rolling_LSTM_RMSE, gbp_rolling_LSTM_MAE, gbp_rolling_LSTM_DA

### Recursive Self-Attention pred

In [ ]:
cad_recursive_Self_Attention_RMSE = root_mean_squared_error(cad_recursive_Self_Attention_pred, cad_groundtruth)
cad_recursive_Self_Attention_MAE = mean_absolute_error(cad_recursive_Self_Attention_pred, cad_groundtruth)
cad_recursive_Self_Attention_DA = direction_accuracy(cad_recursive_Self_Attention_pred, cad_groundtruth)
aud_recursive_Self_Attention_RMSE = root_mean_squared_error(aud_recursive_Self_Attention_pred, aud_groundtruth)
aud_recursive_Self_Attention_MAE = mean_absolute_error(aud_recursive_Self_Attention_pred, aud_groundtruth)
aud_recursive_Self_Attention_DA = direction_accuracy(aud_recursive_Self_Attention_pred, aud_groundtruth)
gbp_recursive_Self_Attention_RMSE = root_mean_squared_error(gbp_recursive_Self_Attention_pred, gbp_groundtruth)
gbp_recursive_Self_Attention_MAE = mean_absolute_error(gbp_recursive_Self_Attention_pred, gbp_groundtruth)
gbp_recursive_Self_Attention_DA = direction_accuracy(gbp_recursive_Self_Attention_pred, gbp_groundtruth)

In [ ]:
cad_recursive_Self_Attention_RMSE, cad_recursive_Self_Attention_MAE, cad_recursive_Self_Attention_DA

In [ ]:
aud_recursive_Self_Attention_RMSE, aud_recursive_Self_Attention_MAE, aud_recursive_Self_Attention_DA

In [ ]:
gbp_recursive_Self_Attention_RMSE, gbp_recursive_Self_Attention_MAE, gbp_recursive_Self_Attention_DA

### Rolling Self-Attention pred

In [ ]:
cad_rolling_Self_Attention_RMSE = root_mean_squared_error(cad_rolling_Self_Attention_pred, cad_groundtruth)
cad_rolling_Self_Attention_MAE = mean_absolute_error(cad_rolling_Self_Attention_pred, cad_groundtruth)
cad_rolling_Self_Attention_DA = direction_accuracy(cad_rolling_Self_Attention_pred, cad_groundtruth)
aud_rolling_Self_Attention_RMSE = root_mean_squared_error(aud_rolling_Self_Attention_pred, aud_groundtruth)
aud_rolling_Self_Attention_MAE = mean_absolute_error(aud_rolling_Self_Attention_pred, aud_groundtruth)
aud_rolling_Self_Attention_DA = direction_accuracy(aud_rolling_Self_Attention_pred, aud_groundtruth)
gbp_rolling_Self_Attention_RMSE = root_mean_squared_error(gbp_rolling_Self_Attention_pred, gbp_groundtruth)
gbp_rolling_Self_Attention_MAE = mean_absolute_error(gbp_rolling_Self_Attention_pred, gbp_groundtruth)
gbp_rolling_Self_Attention_DA = direction_accuracy(gbp_rolling_Self_Attention_pred, gbp_groundtruth)

In [ ]:
cad_rolling_Self_Attention_RMSE, cad_rolling_Self_Attention_MAE, cad_rolling_Self_Attention_DA

In [ ]:
aud_rolling_Self_Attention_RMSE, aud_rolling_Self_Attention_MAE, aud_rolling_Self_Attention_DA

In [ ]:
gbp_rolling_Self_Attention_RMSE, gbp_rolling_Self_Attention_MAE, gbp_rolling_Self_Attention_DA